In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.typing import NDArray
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, train_test_split
from sklearn.multioutput import MultiOutputRegressor
import pandas as pd
from scipy.optimize import minimize
import gurobipy as gp
from gurobipy import GRB
import time


In [2]:
# 価格を生成する関数
def create_price(r_min: float, r_max: float, M: int) -> NDArray[np.float_]:
    # r_minとr_maxの間のランダムな0.1刻みの少数をM個生成
    price = np.random.uniform(r_min, r_max, M)
    price = np.round(price, 1)

    return price


# alphaを作成する関数
def alpha_star(M: int) -> NDArray[np.float_]:
    alpha_star = np.random.uniform(M, 3 * M, size=M)
    return alpha_star


# betaを作成する関数
def beta_star(M: int, M_prime: int) -> NDArray[np.float_]:
    beta_star = np.zeros((M, M_prime))

    for m in range(M):
        for m_prime in range(M_prime):
            if m == m_prime:
                beta_star[m, m_prime] = np.random.uniform(-2 * M, -M)
            else:
                beta_star[m, m_prime] = np.random.uniform(0, 1)

    return beta_star


def quantity_function(
    price: NDArray[np.float_],
    alpha: NDArray[np.float_],
    beta: NDArray[np.float_],
    delta: float = 0.1,  # ノイズレベルを指定（例として0.1を使用）
) -> list[float]:
    M = len(price)
    quantity_list = []
    q_m_no_noise = []

    # ステップ1: ノイズなしのq_mを計算
    for m in range(M):
        sum_beta = 0
        for m_prime in range(M):
            sum_beta += beta[m][m_prime] * price[m_prime]
        quantity = alpha[m] + sum_beta
        q_m_no_noise.append(quantity)

    # E[q_m^2]を計算
    E_q_m_squared = np.mean(np.array(q_m_no_noise) ** 2)

    # ステップ2: ノイズの標準偏差sigmaを計算
    sigma = delta * np.sqrt(E_q_m_squared)

    # ステップ3: ノイズを加えて最終的なq_mを計算
    for m in range(M):
        epsilon = np.random.normal(0, sigma)
        quantity = q_m_no_noise[m] + epsilon
        quantity_list.append(quantity)

    return quantity_list


def sales_function(
    price: NDArray[np.float_], alpha: NDArray[np.float_], beta: NDArray[np.float_]
) -> list[float]:
    M = len(price)
    sales_list = []

    for m in range(M):
        sum_beta = 0
        for m_prime in range(M):
            sum_beta += beta[m][m_prime] * price[m_prime]

        quantity = alpha[m] + sum_beta
        sales_list.append(quantity * price[m])

    return sales_list


In [3]:
# 目的関数を定義
def predict_objective_function(
    prices: NDArray[np.float_], intercepts: [float], coefs: [NDArray[np.float_]], M: int
) -> float:
    # 各変数の内容をデバッグ出力
    # print("prices:", prices)
    # print("intercepts:", intercepts)
    # print("coefs:", coefs)
    # print("M:", M)

    return -sum(
        prices[m]
        * (intercepts[m] + sum(coefs[m][m_prime] * prices[m_prime] for m_prime in range(M)))
        for m in range(M)
    )


In [4]:
# 目的関数を定義（最大化問題を最小化問題に変換）
def sales_objective_function(prices, alpha, beta, M):
    return -sum(
        prices[m] * (alpha[m] + sum(beta[m][m_prime] * prices[m_prime] for m_prime in range(M)))
        for m in range(M)
    )


In [24]:
# 予測と最適化を行う関数
def predict_optimize_BFGS(
    M: int, X: NDArray[np.float_], Y: NDArray[np.float_], prices_list: list[float]
) -> tuple[float, NDArray[np.float_]]:
    lr = MultiOutputRegressor(LinearRegression())
    lr.fit(X, Y)
    # 係数と切片を取得
    coefs = [estimate.coef_ for estimate in lr.estimators_]
    intercepts = [estimate.intercept_ for estimate in lr.estimators_]

    # 初期値として与えられたprices_listを使用
    initial_prices = np.full(M, 0.6)
    # 各価格の範囲を設定（0.6から1.0）
    bounds = [(0.6, 1.0) for _ in range(M)]
    # 最適化を実行
    result = minimize(
        predict_objective_function,
        initial_prices,
        args=(intercepts, coefs, M),
        bounds=bounds,
        method="L-BFGS-B",
    )
    # 最適な価格と目的関数の値を取得
    optimal_prices = result.x
    optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す
    return optimal_value, optimal_prices


# 予測と最適化を行う関数
def predict_optimize_SLSQP(
    M: int, X: NDArray[np.float_], Y: NDArray[np.float_], prices_list: list[float]
) -> tuple[float, NDArray[np.float_]]:
    lr = MultiOutputRegressor(LinearRegression())
    lr.fit(X, Y)
    # 係数と切片を取得
    coefs = [estimate.coef_ for estimate in lr.estimators_]
    intercepts = [estimate.intercept_ for estimate in lr.estimators_]

    # 初期値として与えられたprices_listを使用
    initial_prices = np.full(M, 0.6)
    # 各価格の範囲を設定（0.6から1.0）
    bounds = [(0.6, 1.0) for _ in range(M)]
    # 最適化を実行
    result = minimize(
        predict_objective_function,
        initial_prices,
        args=(intercepts, coefs, M),
        bounds=bounds,
        method="slsqp",
    )
    # 最適な価格と目的関数の値を取得
    optimal_prices = result.x
    optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す
    return optimal_value, optimal_prices


# 予測と最適化を行う関数
def predict_optimize_Neldar(
    M: int, X: NDArray[np.float_], Y: NDArray[np.float_], prices_list: list[float]
) -> tuple[float, NDArray[np.float_]]:
    lr = MultiOutputRegressor(LinearRegression())
    lr.fit(X, Y)
    # 係数と切片を取得
    coefs = [estimate.coef_ for estimate in lr.estimators_]
    intercepts = [estimate.intercept_ for estimate in lr.estimators_]

    # 初期値として与えられたprices_listを使用
    initial_prices = np.full(M, 0.6)
    # 各価格の範囲を設定（0.6から1.0）
    bounds = [(0.6, 1.0) for _ in range(M)]
    # 最適化を実行
    result = minimize(
        predict_objective_function,
        initial_prices,
        args=(intercepts, coefs, M),
        bounds=bounds,
        method="Neldar-Mead",
        options={"adaptive": True},
    )
    # 最適な価格と目的関数の値を取得
    optimal_prices = result.x
    optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す
    return optimal_value, optimal_prices


In [25]:
def sales_optimize_BFGS(
    M: int,
    alpha: np.ndarray,
    beta: np.ndarray,
    prices_list: list[float],
) -> tuple[float, np.ndarray]:
    # 初期値として与えられたprices_listを使用
    initial_prices = np.full(M, 0.6)
    # 各価格の範囲を設定（0.6から1.0）
    bounds = [(0.6, 1.0) for _ in range(M)]
    # 最適化を実行
    result = minimize(
        sales_objective_function,
        initial_prices,
        args=(alpha, beta, M),
        bounds=bounds,
        method="L-BFGS-B",
    )
    # 最適な価格と目的関数の値を取得
    optimal_prices = result.x
    optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す
    return optimal_value, optimal_prices


def sales_optimize_SLSQP(
    M: int,
    alpha: np.ndarray,
    beta: np.ndarray,
    prices_list: list[float],
) -> tuple[float, np.ndarray]:
    # 初期値として与えられたprices_listを使用
    initial_prices = np.full(M, 0.6)
    # 各価格の範囲を設定（0.6から1.0）
    bounds = [(0.6, 1.0) for _ in range(M)]
    # 最適化を実行
    result = minimize(
        sales_objective_function,
        initial_prices,
        args=(alpha, beta, M),
        bounds=bounds,
        method="SLSQP",
    )
    # 最適な価格と目的関数の値を取得
    optimal_prices = result.x
    optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す
    return optimal_value, optimal_prices


def sales_optimize_BFGS(
    M: int,
    alpha: np.ndarray,
    beta: np.ndarray,
    prices_list: list[float],
) -> tuple[float, np.ndarray]:
    # 初期値として与えられたprices_listを使用
    initial_prices = np.full(M, 0.6)
    # 各価格の範囲を設定（0.6から1.0）
    bounds = [(0.6, 1.0) for _ in range(M)]
    # 最適化を実行
    result = minimize(
        sales_objective_function,
        initial_prices,
        args=(alpha, beta, M),
        bounds=bounds,
        method="L-BFGS-B",
    )
    # 最適な価格と目的関数の値を取得
    optimal_prices = result.x
    optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す
    return optimal_value, optimal_prices


def sales_optimize_Neldar(
    M: int,
    alpha: np.ndarray,
    beta: np.ndarray,
    prices_list: list[float],
) -> tuple[float, np.ndarray]:
    # 初期値として与えられたprices_listを使用
    initial_prices = np.full(M, 0.6)
    # 各価格の範囲を設定（0.6から1.0）
    bounds = [(0.6, 1.0) for _ in range(M)]
    # 最適化を実行
    result = minimize(
        sales_objective_function,
        initial_prices,
        args=(alpha, beta, M),
        bounds=bounds,
        method="Neldar-Mead",
        options={"adaptive": True},
    )
    # 最適な価格と目的関数の値を取得
    optimal_prices = result.x
    optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す
    return optimal_value, optimal_prices


In [11]:
# CVを行う関数
def cross_validation_BFGS(
    X: NDArray[np.float_],
    y: NDArray[np.float_],
    M: int,
    K: int,
    prices_list: list[float],
) -> float:
    kf = KFold(n_splits=K, shuffle=True, random_state=0)
    optimal_sales_list = []
    Z = len(prices_list)

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # trainで学習
        lr_tilda = MultiOutputRegressor(LinearRegression())

        # 係数、切片を取得
        lr_tilda.fit(X_train, y_train)
        coefs = [estimate.coef_ for estimate in lr_tilda.estimators_]
        intercepts = [estimate.intercept_ for estimate in lr_tilda.estimators_]

        # 初期値として与えられたprices_listを使用
        initial_prices = np.full(M, 0.6)
        # 各価格の範囲を設定（0.6から1.0）
        bounds = [(0.6, 1.0) for _ in range(M)]
        # 最適化を実行
        result = minimize(
            predict_objective_function,
            initial_prices,
            args=(intercepts, coefs, M),
            bounds=bounds,
            method="BFGS",
        )
        # 最適な価格と目的関数の値を取得
        optimal_prices = result.x
        optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す

        # testで学習
        lr_hat = MultiOutputRegressor(LinearRegression())
        lr_hat.fit(X_test, y_test)

        quantity_hat = lr_hat.predict([optimal_prices])
        sales_hat = np.sum(quantity_hat * optimal_prices)

        optimal_sales_list.append(sales_hat)

    return np.mean(optimal_sales_list)


# CVを行う関数
def cross_validation_SLSQP(
    X: NDArray[np.float_],
    y: NDArray[np.float_],
    M: int,
    K: int,
    prices_list: list[float],
) -> float:
    kf = KFold(n_splits=K, shuffle=True, random_state=0)
    optimal_sales_list = []
    Z = len(prices_list)

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # trainで学習
        lr_tilda = MultiOutputRegressor(LinearRegression())

        # 係数、切片を取得
        lr_tilda.fit(X_train, y_train)
        coefs = [estimate.coef_ for estimate in lr_tilda.estimators_]
        intercepts = [estimate.intercept_ for estimate in lr_tilda.estimators_]

        # 初期値として与えられたprices_listを使用
        initial_prices = np.full(M, 0.6)
        # 各価格の範囲を設定（0.6から1.0）
        bounds = [(0.6, 1.0) for _ in range(M)]
        # 最適化を実行
        result = minimize(
            predict_objective_function,
            initial_prices,
            args=(intercepts, coefs, M),
            bounds=bounds,
            method="SLSQP",
        )
        # 最適な価格と目的関数の値を取得
        optimal_prices = result.x
        optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す

        # testで学習
        lr_hat = MultiOutputRegressor(LinearRegression())
        lr_hat.fit(X_test, y_test)

        quantity_hat = lr_hat.predict([optimal_prices])
        sales_hat = np.sum(quantity_hat * optimal_prices)

        optimal_sales_list.append(sales_hat)

    return np.mean(optimal_sales_list)


# CVを行う関数
def cross_validation_Neldar(
    X: NDArray[np.float_],
    y: NDArray[np.float_],
    M: int,
    K: int,
    prices_list: list[float],
) -> float:
    kf = KFold(n_splits=K, shuffle=True, random_state=0)
    optimal_sales_list = []
    Z = len(prices_list)

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # trainで学習
        lr_tilda = MultiOutputRegressor(LinearRegression())

        # 係数、切片を取得
        lr_tilda.fit(X_train, y_train)
        coefs = [estimate.coef_ for estimate in lr_tilda.estimators_]
        intercepts = [estimate.intercept_ for estimate in lr_tilda.estimators_]

        # 初期値として与えられたprices_listを使用
        initial_prices = np.full(M, 0.6)
        # 各価格の範囲を設定（0.6から1.0）
        bounds = [(0.6, 1.0) for _ in range(M)]
        # 最適化を実行
        result = minimize(
            predict_objective_function,
            initial_prices,
            args=(intercepts, coefs, M),
            bounds=bounds,
            method="Neldar-Mead",
            options={"adaptive": True},
        )
        # 最適な価格と目的関数の値を取得
        optimal_prices = result.x
        optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す

        # testで学習
        lr_hat = MultiOutputRegressor(LinearRegression())
        lr_hat.fit(X_test, y_test)

        quantity_hat = lr_hat.predict([optimal_prices])
        sales_hat = np.sum(quantity_hat * optimal_prices)

        optimal_sales_list.append(sales_hat)

    return np.mean(optimal_sales_list)


In [12]:
def second_objective_single(price, target, intercepts, coefs, M):
    return (
        target
        - sum(
            price[m]
            * (intercepts[m] + sum(coefs[m][m_prime] * price[m_prime] for m_prime in range(M)))
            for m in range(M)
        )
    ) ** 2


In [13]:
def second_objective_multi(prices, target, intercepts_list, coefs_list, M, K):
    obj_list = []
    for k in range(K):
        intercepts = intercepts_list[k]
        coefs = coefs_list[k]
        # 予測販売量の計算
        quantity_hat = intercepts + np.dot(coefs, prices)
        # 売上の計算
        revenue = np.dot(prices, quantity_hat)
        obj_list.append(revenue)
    # 平均売上の計算
    obj = np.mean(obj_list)
    return (target - obj) ** 2


In [18]:
# CVを行う関数
def twostage_single_BFGS(
    X: NDArray[np.float_],
    y: NDArray[np.float_],
    M: int,
    K: int,
    prices_list: list[float],
):
    start = time.time()
    kf = KFold(n_splits=K, shuffle=True, random_state=0)
    optimal_sales_list = []
    Z = len(prices_list)

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # trainで学習
        lr_tilda = MultiOutputRegressor(LinearRegression())

        # 係数、切片を取得
        lr_tilda.fit(X_train, y_train)
        coefs = [estimate.coef_ for estimate in lr_tilda.estimators_]
        intercepts = [estimate.intercept_ for estimate in lr_tilda.estimators_]

        # 初期値として与えられたprices_listを使用
        initial_prices = np.full(M, 0.6)
        # 各価格の範囲を設定（0.6から1.0）
        bounds = [(0.6, 1.0) for _ in range(M)]
        # 最適化を実行
        result = minimize(
            predict_objective_function,
            initial_prices,
            args=(intercepts, coefs, M),
            bounds=bounds,
            method="SLSQP",
        )
        # 最適な価格と目的関数の値を取得
        optimal_prices = result.x
        optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す

        # testで学習
        lr_hat = MultiOutputRegressor(LinearRegression())
        lr_hat.fit(X_test, y_test)

        quantity_hat = lr_hat.predict([optimal_prices])
        sales_hat = np.sum(quantity_hat * optimal_prices)

        optimal_sales_list.append(sales_hat)

    cv_optimal_sales = np.mean(optimal_sales_list)

    # 2段階目の最適化

    lr_second = MultiOutputRegressor(LinearRegression())
    lr_second.fit(X, y)
    # 係数と切片を取得
    coefs_2 = [estimate.coef_ for estimate in lr_second.estimators_]
    intercepts_2 = [estimate.intercept_ for estimate in lr_second.estimators_]

    result_2 = minimize(
        second_objective_single,
        initial_prices,
        args=(cv_optimal_sales, intercepts_2, coefs_2, M),
        bounds=bounds,
        method="L-BFGS-B",
    )

    optimal_prices_2 = result_2.x

    end = time.time()
    run_time = end - start

    return optimal_prices_2, cv_optimal_sales, run_time


In [19]:
# CVを行う関数
def twostage_single_SLSQP(
    X: NDArray[np.float_],
    y: NDArray[np.float_],
    M: int,
    K: int,
    prices_list: list[float],
):
    start = time.time()
    kf = KFold(n_splits=K, shuffle=True, random_state=0)
    optimal_sales_list = []
    Z = len(prices_list)

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # trainで学習
        lr_tilda = MultiOutputRegressor(LinearRegression())

        # 係数、切片を取得
        lr_tilda.fit(X_train, y_train)
        coefs = [estimate.coef_ for estimate in lr_tilda.estimators_]
        intercepts = [estimate.intercept_ for estimate in lr_tilda.estimators_]

        # 初期値として与えられたprices_listを使用
        initial_prices = np.full(M, 0.6)
        # 各価格の範囲を設定（0.6から1.0）
        bounds = [(0.6, 1.0) for _ in range(M)]
        # 最適化を実行
        result = minimize(
            predict_objective_function,
            initial_prices,
            args=(intercepts, coefs, M),
            bounds=bounds,
            method="SLSQP",
        )
        # 最適な価格と目的関数の値を取得
        optimal_prices = result.x
        optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す

        # testで学習
        lr_hat = MultiOutputRegressor(LinearRegression())
        lr_hat.fit(X_test, y_test)

        quantity_hat = lr_hat.predict([optimal_prices])
        sales_hat = np.sum(quantity_hat * optimal_prices)

        optimal_sales_list.append(sales_hat)

    cv_optimal_sales = np.mean(optimal_sales_list)

    # 2段階目の最適化

    lr_second = MultiOutputRegressor(LinearRegression())
    lr_second.fit(X, y)
    # 係数と切片を取得
    coefs_2 = [estimate.coef_ for estimate in lr_second.estimators_]
    intercepts_2 = [estimate.intercept_ for estimate in lr_second.estimators_]

    result_2 = minimize(
        second_objective_single,
        initial_prices,
        args=(cv_optimal_sales, intercepts_2, coefs_2, M),
        bounds=bounds,
        method="SLSQP",
    )

    optimal_prices_2 = result_2.x

    end = time.time()
    run_time = end - start

    return optimal_prices_2, cv_optimal_sales, run_time


In [20]:
# CVを行う関数
def twostage_single_Neldar(
    X: NDArray[np.float_],
    y: NDArray[np.float_],
    M: int,
    K: int,
    prices_list: list[float],
):
    start = time.time()
    kf = KFold(n_splits=K, shuffle=True, random_state=0)
    optimal_sales_list = []
    Z = len(prices_list)

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # trainで学習
        lr_tilda = MultiOutputRegressor(LinearRegression())

        # 係数、切片を取得
        lr_tilda.fit(X_train, y_train)
        coefs = [estimate.coef_ for estimate in lr_tilda.estimators_]
        intercepts = [estimate.intercept_ for estimate in lr_tilda.estimators_]

        # 初期値として与えられたprices_listを使用
        initial_prices = np.full(M, 0.6)
        # 各価格の範囲を設定（0.6から1.0）
        bounds = [(0.6, 1.0) for _ in range(M)]
        # 最適化を実行
        result = minimize(
            predict_objective_function,
            initial_prices,
            args=(intercepts, coefs, M),
            bounds=bounds,
            method="SLSQP",
        )
        # 最適な価格と目的関数の値を取得
        optimal_prices = result.x
        optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す

        # testで学習
        lr_hat = MultiOutputRegressor(LinearRegression())
        lr_hat.fit(X_test, y_test)

        quantity_hat = lr_hat.predict([optimal_prices])
        sales_hat = np.sum(quantity_hat * optimal_prices)

        optimal_sales_list.append(sales_hat)

    cv_optimal_sales = np.mean(optimal_sales_list)

    # 2段階目の最適化

    lr_second = MultiOutputRegressor(LinearRegression())
    lr_second.fit(X, y)
    # 係数と切片を取得
    coefs_2 = [estimate.coef_ for estimate in lr_second.estimators_]
    intercepts_2 = [estimate.intercept_ for estimate in lr_second.estimators_]

    result_2 = minimize(
        second_objective_single,
        initial_prices,
        args=(cv_optimal_sales, intercepts_2, coefs_2, M),
        bounds=bounds,
        method="Neldar-Mead",
        options={"adaptive": True},
    )

    optimal_prices_2 = result_2.x

    end = time.time()
    run_time = end - start

    return optimal_prices_2, cv_optimal_sales, run_time


In [21]:
# CVを行う関数
def twostage_multi_BFGS(
    X: NDArray[np.float_],
    y: NDArray[np.float_],
    M: int,
    K: int,
    prices_list: list[float],
) -> float:
    kf = KFold(n_splits=K, shuffle=True, random_state=0)
    optimal_sales_list = []
    Z = len(prices_list)

    intercepts_list = []
    coefs_list = []

    start = time.time()

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # trainで学習
        lr_tilda = MultiOutputRegressor(LinearRegression())

        # 係数、切片を取得
        lr_tilda.fit(X_train, y_train)
        coefs = [estimate.coef_ for estimate in lr_tilda.estimators_]
        intercepts = [estimate.intercept_ for estimate in lr_tilda.estimators_]

        intercepts_list.append(intercepts)
        coefs_list.append(coefs)

        # 初期値として与えられたprices_listを使用
        initial_prices = np.full(M, 0.6)
        # 各価格の範囲を設定（0.6から1.0）
        bounds = [(0.6, 1.0) for _ in range(M)]
        # 最適化を実行
        result = minimize(
            predict_objective_function,
            initial_prices,
            args=(intercepts, coefs, M),
            bounds=bounds,
            method="SLSQP",
        )
        # 最適な価格と目的関数の値を取得
        optimal_prices = result.x
        optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す

        # testで学習
        lr_hat = MultiOutputRegressor(LinearRegression())
        lr_hat.fit(X_test, y_test)

        quantity_hat = lr_hat.predict([optimal_prices])
        sales_hat = np.sum(quantity_hat * optimal_prices)

        optimal_sales_list.append(sales_hat)

    cv_optimal_sales = np.mean(optimal_sales_list)

    # 2段階目の最適化

    result_2 = minimize(
        second_objective_multi,
        initial_prices,
        args=(cv_optimal_sales, intercepts_list, coefs_list, M, K),
        bounds=bounds,
        method="L-BFGS-B",
    )

    optimal_prices_2 = result_2.x

    # 関数の実行時間を計測

    end = time.time()

    run_time = end - start

    return optimal_prices_2, cv_optimal_sales, run_time


In [22]:
# CVを行う関数
def twostage_multi_SLSQP(
    X: NDArray[np.float_],
    y: NDArray[np.float_],
    M: int,
    K: int,
    prices_list: list[float],
) -> float:
    kf = KFold(n_splits=K, shuffle=True, random_state=0)
    optimal_sales_list = []
    Z = len(prices_list)

    intercepts_list = []
    coefs_list = []

    start = time.time()

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # trainで学習
        lr_tilda = MultiOutputRegressor(LinearRegression())

        # 係数、切片を取得
        lr_tilda.fit(X_train, y_train)
        coefs = [estimate.coef_ for estimate in lr_tilda.estimators_]
        intercepts = [estimate.intercept_ for estimate in lr_tilda.estimators_]

        intercepts_list.append(intercepts)
        coefs_list.append(coefs)

        # 初期値として与えられたprices_listを使用
        initial_prices = np.full(M, 0.6)
        # 各価格の範囲を設定（0.6から1.0）
        bounds = [(0.6, 1.0) for _ in range(M)]
        # 最適化を実行
        result = minimize(
            predict_objective_function,
            initial_prices,
            args=(intercepts, coefs, M),
            bounds=bounds,
            method="SLSQP",
        )
        # 最適な価格と目的関数の値を取得
        optimal_prices = result.x
        optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す

        # testで学習
        lr_hat = MultiOutputRegressor(LinearRegression())
        lr_hat.fit(X_test, y_test)

        quantity_hat = lr_hat.predict([optimal_prices])
        sales_hat = np.sum(quantity_hat * optimal_prices)

        optimal_sales_list.append(sales_hat)

    cv_optimal_sales = np.mean(optimal_sales_list)

    # 2段階目の最適化

    result_2 = minimize(
        second_objective_multi,
        initial_prices,
        args=(cv_optimal_sales, intercepts_list, coefs_list, M, K),
        bounds=bounds,
        method="SLSQP",
    )

    optimal_prices_2 = result_2.x

    # 関数の実行時間を計測

    end = time.time()

    run_time = end - start

    return optimal_prices_2, cv_optimal_sales, run_time


In [23]:
# CVを行う関数
def twostage_multi_Neldar(
    X: NDArray[np.float_],
    y: NDArray[np.float_],
    M: int,
    K: int,
    prices_list: list[float],
) -> float:
    kf = KFold(n_splits=K, shuffle=True, random_state=0)
    optimal_sales_list = []
    Z = len(prices_list)

    intercepts_list = []
    coefs_list = []

    start = time.time()

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # trainで学習
        lr_tilda = MultiOutputRegressor(LinearRegression())

        # 係数、切片を取得
        lr_tilda.fit(X_train, y_train)
        coefs = [estimate.coef_ for estimate in lr_tilda.estimators_]
        intercepts = [estimate.intercept_ for estimate in lr_tilda.estimators_]

        intercepts_list.append(intercepts)
        coefs_list.append(coefs)

        # 初期値として与えられたprices_listを使用
        initial_prices = np.full(M, 0.6)
        # 各価格の範囲を設定（0.6から1.0）
        bounds = [(0.6, 1.0) for _ in range(M)]
        # 最適化を実行
        result = minimize(
            predict_objective_function,
            initial_prices,
            args=(intercepts, coefs, M),
            bounds=bounds,
            method="SLSQP",
        )
        # 最適な価格と目的関数の値を取得
        optimal_prices = result.x
        optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す

        # testで学習
        lr_hat = MultiOutputRegressor(LinearRegression())
        lr_hat.fit(X_test, y_test)

        quantity_hat = lr_hat.predict([optimal_prices])
        sales_hat = np.sum(quantity_hat * optimal_prices)

        optimal_sales_list.append(sales_hat)

    cv_optimal_sales = np.mean(optimal_sales_list)

    # 2段階目の最適化

    result_2 = minimize(
        second_objective_multi,
        initial_prices,
        args=(cv_optimal_sales, intercepts_list, coefs_list, M, K),
        bounds=bounds,
        method="Neldar-Mead",
        options={"adaptive": True},
    )

    optimal_prices_2 = result_2.x

    # 関数の実行時間を計測

    end = time.time()

    run_time = end - start

    return optimal_prices_2, cv_optimal_sales, run_time


In [ ]:
# M =10 で100回実験

# 結果を格納するリスト
predict_optimize_BFGS_value_list_10 = []

predict_optimize_SLSQP_value_list_10 = []

predict_optimize_Neldar_value_list_10 = []

sales_optimize_BFGS_value_list_10 = []

sales_optimize_SLSQP_value_list_10 = []

sales_optimize_Neldar_value_list_10 = []

po_prices_BFGS_list_10 = []
po_prices_SLSQP_list_10 = []
po_prices_Neldar_list_10 = []

twostage_single_BFGS_value_list_10 = []
twostage_single_BFGS_runtime_list_10 = []

twostage_single_SLSQP_value_list_10 = []
twostage_single_SLSQP_runtime_list_10 = []

twostage_single_Neldar_value_list_10 = []
twostage_single_Neldar_runtime_list_10 = []

ts_BFGS_list_10 = []
ts_SLSQP_list_10 = []
ts_Neldar_list_10 = []

twostage_multi_BFGS_value_list_10 = []
twostage_multi_BFGS_runtime_list_10 = []

twostage_multi_SLSQP_value_list_10 = []
twostage_multi_SLSQP_runtime_list_10 = []

twostage_multi_Neldar_value_list_10 = []
twostage_multi_Neldar_runtime_list_10 = []

tm_BFGS_list_10 = []
tm_SLSQP_list_10 = []
tm_Neldar_list_10 = []

M = 10
K = 5
for i in range(100):
    # データの生成
    alpha = alpha_star(M)
    beta = beta_star(M, M)

    r_m = 0.6
    r_M = 1.0

    prices_list = []
    quantity_list = []

    for _ in range(500):
        prices = create_price(M, r_m, r_M)
        prices_list.append(prices)
        quantity = quantity_function(prices, alpha, beta)
        quantity_list.append(quantity)

    X = np.array(prices_list).astype(float)
    y = np.array(quantity_list).astype(float)

    z = [0.6, 0.7, 0.8, 0.9, 1.0]

    # 予測と最適化を行う関数
    predict_optimize_BFGS_value, predict_optimize_BFGS_prices = predict_optimize_BFGS(M, X, y, z)
    predict_optimize_SLSQP_value, predict_optimize_SLSQP_prices = predict_optimize_SLSQP(
        M, X, y, z
    )
    predict_optimize_Neldar_value, predict_optimize_Neldar_prices = predict_optimize_Neldar(
        M, X, y, z
    )

    # 販売と最適化を行う関数
    sales_optimize_BFGS_value, sales_optimize_BFGS_prices = sales_optimize_BFGS(M, alpha, beta, z)
    sales_optimize_SLSQP_value, sales_optimize_SLSQP_prices = sales_optimize_SLSQP(
        M, alpha, beta, z
    )
    sales_optimize_Neldar_value, sales_optimize_Neldar_prices = sales_optimize_Neldar(
        M, alpha, beta, z
    )

    # 予測と最適化を行った価格をsales_functionに代入
    po_prices_BFGS = sales_function(predict_optimize_BFGS_prices, alpha, beta)
    po_prices_SLSQP = sales_function(predict_optimize_SLSQP_prices, alpha, beta)
    po_prices_Neldar = sales_function(predict_optimize_Neldar_prices, alpha, beta)

    # 2段階最適化を行う関数(single)
    twostage_single_BFGS_prices, twostage_single_BFGS_value, twostage_single_BFGS_runtime = (
        twostage_single_BFGS(X, y, M, K, z)
    )
    twostage_single_SLSQP_prices, twostage_single_SLSQP_value, twostage_single_SLSQP_runtime = (
        twostage_single_SLSQP(X, y, M, K, z)
    )
    twostage_single_Neldar_prices, twostage_single_Neldar_value, twostage_single_Neldar_runtime = (
        twostage_single_Neldar(X, y, M, K, z)
    )

    # 2段階最適化を行った価格をsales_functionに代入
    ts_BFGS = sales_function(twostage_single_BFGS_prices, alpha, beta)
    ts_SLSQP = sales_function(twostage_single_SLSQP_prices, alpha, beta)
    ts_Neldar = sales_function(twostage_single_Neldar_prices, alpha, beta)

    # 2段階最適化を行う関数(multi)
    twostage_multi_BFGS_prices, twostage_multi_BFGS_value, twostage_multi_BFGS_runtime = (
        twostage_multi_BFGS(X, y, M, K, z)
    )
    twostage_multi_SLSQP_prices, twostage_multi_SLSQP_value, twostage_multi_SLSQP_runtime = (
        twostage_multi_SLSQP(X, y, M, K, z)
    )
    twostage_multi_Neldar_prices, twostage_multi_Neldar_value, twostage_multi_Neldar_runtime = (
        twostage_multi_Neldar(X, y, M, K, z)
    )

    # 2段階最適化を行った価格をsales_functionに代入
    tm_BFGS = sales_function(twostage_multi_BFGS_prices, alpha, beta)
    tm_SLSQP = sales_function(twostage_multi_SLSQP_prices, alpha, beta)
    tm_Neldar = sales_function(twostage_multi_Neldar_prices, alpha, beta)

    # 結果を格納
    predict_optimize_BFGS_value_list_10.append(
        predict_optimize_BFGS_value / sales_optimize_BFGS_value
    )

    predict_optimize_SLSQP_value_list_10.append(
        predict_optimize_SLSQP_value / sales_optimize_SLSQP_value
    )
    

    predict_optimize_Neldar_value_list_10.append(
        predict_optimize_Neldar_value / sales_optimize_Neldar_value
    )

    sales_optimize_BFGS_value_list_10.append(sales_optimize_BFGS_value / sales_optimize_BFGS_value)
    

    sales_optimize_SLSQP_value_list_10.append(
        sales_optimize_SLSQP_value / sales_optimize_SLSQP_value
    )
    

    sales_optimize_Neldar_value_list_10.append(
        sales_optimize_Neldar_value / sales_optimize_Neldar_value
    )
    
    po_prices_BFGS_list_10.append(po_prices_BFGS/sales_optimize_BFGS_value)

    po_prices_SLSQP_list_10.append(po_prices_SLSQP/sales_optimize_SLSQP_value)

    po_prices_Neldar_list_10.append(po_prices_Neldar/sales_optimize_Neldar_value)

    twostage_single_BFGS_value_list_10.append(twostage_single_BFGS_value / sales_optimize_BFGS_value)

    twostage_single_BFGS_runtime_list_10.append(twostage_single_BFGS_runtime)

    twostage_single_SLSQP_value_list_10.append(twostage_single_SLSQP_value / sales_optimize_SLSQP_value)

    twostage_single_SLSQP_runtime_list_10.append(twostage_single_SLSQP_runtime)

    twostage_single_Neldar_value_list_10.append(twostage_single_Neldar_value / sales_optimize_Neldar_value)

    twostage_single_Neldar_runtime_list_10.append(twostage_single_Neldar_runtime)

    ts_BFGS_list_10.append(ts_BFGS/sales_optimize_BFGS_value)

    ts_SLSQP_list_10.append(ts_SLSQP/sales_optimize_SLSQP_value)

    ts_Neldar_list_10.append(ts_Neldar/sales_optimize_Neldar_value)

    twostage_multi_BFGS_value_list_10.append(twostage_multi_BFGS_value / sales_optimize_BFGS_value)

    twostage_multi_BFGS_runtime_list_10.append(twostage_multi_BFGS_runtime)

    twostage_multi_SLSQP_value_list_10.append(twostage_multi_SLSQP_value / sales_optimize_SLSQP_value)

    twostage_multi_SLSQP_runtime_list_10.append(twostage_multi_SLSQP_runtime)

    twostage_multi_Neldar_value_list_10.append(twostage_multi_Neldar_value / sales_optimize_Neldar_value)

    twostage_multi_Neldar_runtime_list_10.append(twostage_multi_Neldar_runtime)

    


TypeError: expected a sequence of integers or a single integer, got '1.0'